In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v3.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [5]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
85144,Tammy_Abraham,30,4,10,Chelsea,Aston Villa,1.707847,0.565630,False,1,...,2,1,4.0,121926,211365,89439,0,2020-06-21T15:15:00Z,1920,1.0
85145,Nicolás_Otamendi,30,2,0,Manchester City,Arsenal,2.569328,1.439201,True,0,...,0,3,0.0,35112,66925,31813,0,2020-06-17T19:15:00Z,1920,1.0
85146,Nicolás_Otamendi,30,2,90,Manchester City,Burnley,2.569328,0.399883,True,6,...,0,5,0.0,35112,66925,31813,0,2020-06-22T19:00:00Z,1920,1.0
85147,James_Daly,30,4,0,Crystal Palace,Bournemouth,0.437038,0.695809,False,0,...,2,0,0.0,-15,30,45,0,2020-06-20T18:45:00Z,1920,0.0
85148,Declan_Rice,30,3,90,West Ham United,Wolverhampton Wanderers,0.737926,0.750647,True,2,...,2,0,33.0,-4399,6328,10727,0,2020-06-20T16:30:00Z,1920,1.0
85149,Connor_Wickham,30,4,0,Crystal Palace,Bournemouth,0.437038,0.695809,False,0,...,2,0,0.0,-3409,230,3639,0,2020-06-20T18:45:00Z,1920,0.0
85150,Mark_Noble,30,3,90,West Ham United,Wolverhampton Wanderers,0.737926,0.750647,True,2,...,2,0,0.0,-21087,7154,28241,0,2020-06-20T16:30:00Z,1920,1.0
85151,Mark_Travers,30,1,0,Bournemouth,Crystal Palace,0.695809,0.437038,True,0,...,2,0,0.0,-344,342,686,0,2020-06-20T18:45:00Z,1920,1.0
85152,Theo_Walcott,30,3,0,Everton,Liverpool,1.060100,2.459124,True,0,...,0,0,0.0,-846,119,965,0,2020-06-21T18:00:00Z,1920,0.0
85153,Alisson_Ramses Becker,30,1,90,Liverpool,Everton,2.459124,1.060100,False,10,...,0,0,0.0,-19352,24729,44081,0,2020-06-21T18:00:00Z,1920,1.0


In [6]:
# df_train[(df_train['gw']==24) & (df_train['season']=='1920') & (df_train['team']=='Liverpool')]

In [7]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

31

In [8]:
# last_gw = 28

In [9]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [10]:
df_latest.drop_duplicates(inplace=True)

In [11]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Chris_Mepham,31,2,0,Bournemouth,Wolverhampton Wanderers,0.684948,0.750614,False,0,...,46355,0,1,0.0,-99,43,142,0,2020-06-24T17:00:00Z,1920
1,Paulo_Gazzaniga,31,1,0,Tottenham Hotspur,West Ham United,1.765705,0.734871,True,0,...,125503,0,2,0.0,-5025,1832,6857,0,2020-06-23T19:15:00Z,1920
2,John_Egan,31,2,0,Sheffield United,Manchester United,0.341051,1.632156,False,0,...,253023,0,3,0.0,-99129,8705,107834,0,2020-06-24T17:00:00Z,1920
3,Max_Meyer,31,3,75,Crystal Palace,Liverpool,0.436389,2.459015,False,2,...,5978,0,4,1.0,-36,16,52,0,2020-06-24T19:15:00Z,1920
4,Fabian_Delph,31,2,0,Everton,Norwich,1.058290,0.335837,False,0,...,13158,1,0,0.0,-134,10,144,0,2020-06-24T17:00:00Z,1920
5,James_Justin,31,2,90,Leicester City,Brighton and Hove Albion,1.024285,0.463214,True,5,...,37903,0,0,4.0,9339,16065,6726,1,2020-06-23T17:00:00Z,1920
6,Che_Adams,31,4,0,Southampton,Arsenal,0.483617,1.459289,True,0,...,23737,2,0,0.0,-78,245,323,0,2020-06-25T17:00:00Z,1920
7,Mahmoud Ahmed_Ibrahim Hassan,31,3,69,Aston Villa,Newcastle United,0.557545,0.629911,False,2,...,80711,1,1,8.0,-2087,414,2501,0,2020-06-24T17:00:00Z,1920
8,Tyreece_John-Jules,31,4,0,Arsenal,Southampton,1.459289,0.483617,False,0,...,401,2,0,0.0,4,29,25,0,2020-06-25T17:00:00Z,1920
9,Adrian_Mariappa,31,2,0,Watford,Burnley,0.481224,0.399865,False,0,...,19817,0,1,0.0,-117,343,460,0,2020-06-25T17:00:00Z,1920


In [12]:
df_latest.shape

(644, 34)

In [13]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [14]:
last_play_proba.head()

,player,play_proba
0,Ben_Mee,1.0
1,James_Tarkowski,1.0
2,Matthew_Lowton,1.0
3,Phil_Bardsley,1.0
4,Charlie_Taylor,1.0


In [15]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [16]:
df_latest.drop_duplicates(inplace=True)

In [17]:
df_train.drop_duplicates(inplace=True)

In [18]:
df_train.shape

(85194, 35)

In [19]:
df_latest.shape

(644, 35)

In [20]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [21]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
85833,Tommy_Doyle,31,3,0,Manchester City,Chelsea,2.594403,1.707772,False,0,...,1,2,0.0,0,0,0,0,2020-06-25T19:15:00Z,1920,NaN
85834,Joseph_Anang,31,1,0,West Ham United,Tottenham Hotspur,0.734871,1.765705,False,0,...,0,2,0.0,-97,212,309,0,2020-06-23T19:15:00Z,1920,1.0
85835,Erik_Pieters,31,2,9,Burnley,Watford,0.399865,0.481224,True,1,...,0,1,0.0,-783,1637,2420,0,2020-06-25T17:00:00Z,1920,1.0
85836,Japhet_Tanganga,31,2,0,Tottenham Hotspur,West Ham United,1.765705,0.734871,True,0,...,0,2,0.0,-13777,410,14187,0,2020-06-23T19:15:00Z,1920,0.0
85837,Ravel_Morrison,31,3,0,Sheffield United,Manchester United,0.341051,1.632156,False,0,...,0,3,0.0,-48,7,55,0,2020-06-24T17:00:00Z,1920,0.0


In [22]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v3.csv')

In [23]:
# df_train_new = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

In [24]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [25]:
fixtures.head(20)

,home_team,away_team,gw
310,Arsenal,Norwich,32
311,Aston Villa,Wolverhampton Wanderers,32
312,Bournemouth,Newcastle United,32
313,Brighton and Hove Albion,Manchester United,32
314,Crystal Palace,Burnley,32
315,Everton,Leicester City,32
316,Manchester City,Liverpool,32
317,Sheffield United,Tottenham Hotspur,32
318,Watford,Southampton,32
319,West Ham United,Chelsea,32


In [26]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [27]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,32,Arsenal,Norwich,3,45
1,32,Aston Villa,Wolverhampton Wanderers,7,39
2,32,Bournemouth,Newcastle United,91,4
3,32,Brighton and Hove Albion,Manchester United,36,1
4,32,Crystal Palace,Burnley,31,90


In [28]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [29]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [30]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,32,Norwich,Arsenal,Timm_Klose,2,43,100,False
1,32,Norwich,Arsenal,Ben_Godfrey,2,44,100,False
2,32,Norwich,Arsenal,Maximillian_Aarons,2,43,100,False
3,32,Norwich,Arsenal,Christoph_Zimmermann,2,44,0,False
4,32,Norwich,Arsenal,Jamal_Lewis,2,44,100,False


In [31]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,32,Arsenal,Norwich,Shkodran_Mustafi,2,51,100,True
1,32,Arsenal,Norwich,Héctor_Bellerín,2,54,100,True
2,32,Arsenal,Norwich,Sead_Kolasinac,2,52,100,True
3,32,Arsenal,Norwich,Ainsley_Maitland-Niles,2,45,100,True
4,32,Arsenal,Norwich,Sokratis_Papastathopoulos,2,48,100,True


In [32]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 23.0,
 'Aaron_Cresswell': 90.0,
 'Aaron_Lennon': 1.8,
 'Aaron_Mooy': 80.6,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 90.0,
 'Abd-Al-Ali Morakinyo Olaposi_Koiki': 0.0,
 'Abdoulaye_Doucouré': 75.6,
 'Adam_Idah': 4.6,
 'Adam_Lallana': 7.0,
 'Adam_Masina': 89.4,
 'Adam_Smith': 90.0,
 'Adam_Webster': 76.6,
 'Adama_Traoré': 47.25,
 'Adrian_Mariappa': 2.6,
 'Adrián_San Miguel del Castillo': 18.0,
 'Ahmed_El Mohamady': 17.8,
 'Ainsley_Maitland-Niles': 6.6,
 'Albian_Ajeti': 0.0,
 'Alex_Iwobi': 47.0,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 50.2,
 'Alexander_Tettey': 64.6,
 'Alexandre Nascimento_Costa Silva': 0.0,
 'Alexandre_Lacazette': 28.4,
 'Alexis_Mac Allister': 13.6,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 4.0,
 'Alisson_Ramses Becker': 90.0,
 'Allan_Saint-Maximin': 72.0,
 'Andre_Gray': 2.2,
 'Andreas_Christensen': 90.0,
 'Andreas_Pereira': 2.0,
 'Andrew_Robertson': 52.6,
 'Andrew_Surman': 18.0,
 'Andriy_Yarmolenko': 4.5,
 'Andros_Townsend': 20.8,

In [33]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [34]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,32,Arsenal,Norwich,Shkodran_Mustafi,2,5.1,1.00,True,1.459289,0.335837,1920,90.00
1,32,Arsenal,Norwich,Héctor_Bellerín,2,5.4,1.00,True,1.459289,0.335837,1920,72.20
2,32,Arsenal,Norwich,Sead_Kolasinac,2,5.2,1.00,True,1.459289,0.335837,1920,31.75
3,32,Arsenal,Norwich,Ainsley_Maitland-Niles,2,4.5,1.00,True,1.459289,0.335837,1920,6.60
4,32,Arsenal,Norwich,Sokratis_Papastathopoulos,2,4.8,1.00,True,1.459289,0.335837,1920,30.00
5,32,Arsenal,Norwich,Nacho_Monreal,2,5.0,0.00,True,1.459289,0.335837,1920,0.00
6,32,Arsenal,Norwich,Laurent_Koscielny,2,5.0,0.00,True,1.459289,0.335837,1920,0.00
7,32,Arsenal,Norwich,Konstantinos_Mavropanos,2,4.3,0.00,True,1.459289,0.335837,1920,0.00
8,32,Arsenal,Norwich,Carl_Jenkinson,2,4.5,0.00,True,1.459289,0.335837,1920,0.00
9,32,Arsenal,Norwich,Rob_Holding,2,4.4,1.00,True,1.459289,0.335837,1920,36.00


In [35]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')